In [26]:
#create container
import boto3
import time
import pprint
pp = pprint.PrettyPrinter(depth=4)
ec2 = boto3.resource('ec2')
ec2_client = boto3.client('ec2')
autoscaling = boto3.client('autoscaling')
ecs = boto3.client('ecs')
r53 = boto3.client('route53')
launchtemplate = 'EC2ContainerService-sysvis-EcsInstanceLc-1H1654961ZY8F'
autoscalinggroup = 'EC2ContainerService-sysvis-EcsInstanceAsg-196WYOOZAFI87'
vpc = 'vpc-0d7c412110e5f49f2'
subnet = 'subnet-0596881f608fd4f70'
securitygroup = 'sg-05c4123649a439051'
hosted_zone_id = "Z09241041ZJLPUFRNRNGX"
instance_id = ec2.create_instances(MinCount=1, MaxCount=1, LaunchTemplate={'LaunchTemplateName':launchtemplate}, SecurityGroupIds=[securitygroup], SubnetId=subnet)
instance_id = instance_id[0].instance_id
print('instance id', instance_id)
ready = False
time.sleep(2)
while not ready:
    if ec2_client.describe_instances(InstanceIds=[instance_id])['Reservations'][0]['Instances'][0]['State']['Name'] =='running':
        ready = True
time.sleep(2)
autoscaling.attach_instances(AutoScalingGroupName=autoscalinggroup, InstanceIds=[instance_id])
ready = False
print('created')
while not ready:
    response = autoscaling.describe_auto_scaling_groups(AutoScalingGroupNames=[autoscalinggroup])
    realresponse = response['AutoScalingGroups'][0]
    instances = realresponse['Instances']
    for i in instances:
        if i['InstanceId'] == instance_id:
            ready = True
print('attached')
ready = False
while not ready:
    containers = ecs.list_container_instances(cluster='sysvis')['containerInstanceArns']
    for i in containers:
        container = i
        findinstanceid = ecs.describe_container_instances(cluster='sysvis',containerInstances=[container])['containerInstances'][0]['ec2InstanceId']
        if findinstanceid == instance_id:
            ready = True
task = ecs.start_task(cluster='sysvis', taskDefinition='sysvis', containerInstances=[container])
time.sleep(2)
task_arn = task['tasks'][0]['taskArn']
ready = False
print('container')
while not ready:
    status = ecs.describe_tasks(cluster = 'sysvis', tasks = [task_arn])['tasks'][0]['lastStatus']
    if status == 'RUNNING':
        ready = True
print('finished')
print('Ip Address', ec2_client.describe_instances(InstanceIds=[instance_id])['Reservations'][0]['Instances'][0]['PublicIpAddress'])
ip_address = ec2_client.describe_instances(InstanceIds=[instance_id])['Reservations'][0]['Instances'][0]['PublicIpAddress']
print('Public Dns Name', ec2_client.describe_instances(InstanceIds=[instance_id])['Reservations'][0]['Instances'][0]['PublicDnsName'])
url = instance_id[2:] + ".synesthesiavisualizer.com"
#hosted_zone = r53.create_hosted_zone(Name = url, CallerReference = instance_id[2:])
record = r53.change_resource_record_sets(HostedZoneId=url, ChangeBatch={'Changes': [{'Action': 'CREATE','ResourceRecordSet': {'Name': url,'Type': 'A','ResourceRecords': [{'Value': ip_address},],}},]})
print('zone')
#record_set = r53.change_resource_record_sets(HostedZoneId = url, ChangeBatch = {'Changes': [{}]})

instance id i-03489340862ef9da2


created


attached


container


finished
Ip Address 54.226.79.206


Public Dns Name ec2-54-226-79-206.compute-1.amazonaws.com


NameError: name 'client' is not defined

In [32]:
url = instance_id[2:] + ".synesthesiavisualizer.com"
record = r53.change_resource_record_sets(HostedZoneId=hosted_zone_id, ChangeBatch={'Changes': [{'Action': 'CREATE','ResourceRecordSet': {'Name': url,'Type': 'A','ResourceRecords': [{'Value': ip_address},],}},]})

InvalidInput: An error occurred (InvalidInput) when calling the ChangeResourceRecordSets operation: Invalid request: Expected exactly one of [AliasTarget, all of [TTL, and ResourceRecords], or TrafficPolicyInstanceId], but found none in Change with [Action=CREATE, Name=03489340862ef9da2.synesthesiavisualizer.com, Type=A, SetIdentifier=null]

In [33]:
#delete container
import boto3
autoscaling = boto3.client('autoscaling')
ec2_client = boto3.client('ec2')
#instance_id = 'i-070e63cb007c6703a'
autoscalinggroup = 'EC2ContainerService-sysvis-EcsInstanceAsg-196WYOOZAFI87'
autoscaling = boto3.client('autoscaling')
response = autoscaling.detach_instances(InstanceIds=[instance_id],AutoScalingGroupName=autoscalinggroup, ShouldDecrementDesiredCapacity=True)
print(response)
response = ec2_client.terminate_instances(InstanceIds=[instance_id]) 

{'Activities': [{'ActivityId': '29d5d7d8-ab04-b7e0-d051-31b1b93bac0d', 'AutoScalingGroupName': 'EC2ContainerService-sysvis-EcsInstanceAsg-196WYOOZAFI87', 'Description': 'Detaching EC2 instance: i-03489340862ef9da2', 'Cause': 'At 2020-11-23T17:33:18Z instance i-03489340862ef9da2 was detached in response to a user request, shrinking the capacity from 1 to 0.', 'StartTime': datetime.datetime(2020, 11, 23, 17, 33, 18, 509000, tzinfo=tzutc()), 'StatusCode': 'InProgress', 'Progress': 50, 'Details': '{"Subnet ID":"subnet-0596881f608fd4f70","Availability Zone":"us-east-1a"}'}], 'ResponseMetadata': {'RequestId': '0baf0d30-c27b-4ba4-9054-a76fedcbc44e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0baf0d30-c27b-4ba4-9054-a76fedcbc44e', 'content-type': 'text/xml', 'content-length': '1023', 'date': 'Mon, 23 Nov 2020 17:33:17 GMT'}, 'RetryAttempts': 0}}
